In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os

In [3]:
df_transactions = pd.read_csv("C:/Users/felix.oechslein/Desktop/Analysen Elphi, Mai/verarbeitete_daten/"
                                    "Elphi_transactions_ersteBereinigung.csv")

In [4]:
df_privatkunden_iids = pd.read_csv("C:/Users/felix.oechslein/Desktop/Daten, 13.12.22/" +
                                    "privatkunden.iid.ELPHI.csv", index_col = [0])

##### Privatkunden und Trans mergen

In [5]:
# Privatkunden iids mit Transaktionsdaten mergen
df_t = pd.merge(df_transactions, df_privatkunden_iids, on = ["iid"])
df_t.head()

,iid,date,month_count
0,163094976,2021-06-04 16:33:00,-10
1,163094976,2021-06-04 19:54:00,-10
2,163094976,2021-09-28 12:01:00,-7
3,163094976,2021-10-28 17:43:00,-6
4,163094976,2021-12-21 18:20:00,-4


##### Auswahl der Kunden: Transaktion im April 2022 und mindestes eine Transaktion nach August 2022

In [6]:
# Alle "iid" mit "month_count" = 0
iid_month_0 = df_t.groupby("iid").filter(lambda x: (x["month_count"] == 0).any())["iid"].unique()

# Alle "iid" mit "month_count" >= 7
iid_month_7 = df_t.groupby("iid").filter(lambda x: (x["month_count"] >= 7).any())["iid"].unique()

# Die Schnittmenge beider Listen gibt alle "iid" aus, die beide Bedingungen erfüllen
iid_both = list(set(iid_month_0) & set(iid_month_7))

##### Mergen der Auswahl mit Trans_df

In [7]:
# Annahme: iid_both ist eine Liste von eindeutigen iids
df_t_filtered = df_t[df_t['iid'].isin(iid_both)]

##### Filter nach Trans nach t = -1

In [8]:
# Ausschließlich Transaktionen nach t > -1 relevant
df_t_filtered = df_t_filtered[df_t_filtered["month_count"] > -1]

#### 2. verarbeiteteDaten als Elphi_transactions_zweiteBereinigug.csv in dem Ordner verarbeitete_Daten abspeichern

In [9]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Elphi, Mai/verarbeitete_daten/'
filename = 'Elphi_transactions_zweiteBereinigung.csv'

df_t_filtered.to_csv(os.path.join(path, filename), index=False)

##### 3. Relevante iids als Elphi_transactions_zweiteBereinigug_iids.csv in dem Ordner verarbeitete_Daten abspeichern

In [11]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Elphi, Mai/verarbeitete_daten/'
filename = 'Elphi_transactions_zweiteBereinigung_iids.csv'

df_iid_both = pd.DataFrame(iid_both, columns=["iid"])
df_iid_both.to_csv(os.path.join(path, filename), index=False)